# Mask creation

This notebook uses the sparse pixel annotations (always in the `annotations.csv` file) 
from the CoralNet website, and creates a **mask** for each image.

The annotations are given in the following order: `row, column, label` -> `height, width, label`.

The image shapes are in the same order: `height, width, channels`.

! quick reminder !

numpy image: H x W x C

torch image: C x H x W

To create the masks, the `labelset.csv` is going to be used.

*I am going to use all the labels provided by the sources (even if they are not in the "verified" labels), 
because the images have been selected to the "confirmed".*

## Import necessary libraries

In [1]:
from PIL import Image
from skimage import io, transform
from sklearn.preprocessing import MinMaxScaler  

import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import torch

## Get data from the chosen source

In [6]:
# go to the source
os.chdir("/data/jantina/CoralNet/WAPA_Interns/other")

# prepare annotations
annotations = pd.read_csv('annotations.csv')
img_names = annotations.iloc[:, 0]
landmarks = annotations.iloc[:, 1:4]

# prepare labels as dictionary
# using numbers from 1 -> N
labels = pd.read_csv('labelset.csv')["Short Code"]
labels.index = labels.index + 1
labels = labels.to_dict()

print(f"[INFO] There are {len(img_names.unique())} images in this source")
print(f"[INFO] There are {len(labels)} labels in this source")

[INFO] There are 890 images in this source
[INFO] There are 137 labels in this source


## Create masks for all images in that source

In [ ]:
mask = np.full((height, width), 255)

In [7]:
for img_name in img_names.unique():
    # go to root
    os.chdir("/data/jantina/CoralNet/WAPA_Interns")
    # get image
    try:
        image = io.imread(os.path.join('images', img_name))
        # retrieve the annotations for that image
        landmark = landmarks.iloc[img_names[img_names==img_name].index.values]
        landmark = np.asarray(landmark)

        # get image information
        height = image.shape[0]
        width = image.shape[1]
        colors = image.shape[2]

        # create array with coordinates and label code
        classes = np.copy(landmark) 
        for new, old in labels.items():
            classes[landmark == old] = new

        # create mask 
        mask = np.full((height, width), 255)
        for x in classes:
            try:
                mask[x[0], x[1]] = x[2]
            except:
                print(img_name)
                print("These labels are out of bounds", x[0], x[1], x[2])
                continue

        # switch to png to not loose information
        img_name = img_name.split(".JPG")[0]+".png"

        # check the labels contained in the mask
        print(img_name, np.unique(mask))

        # save mask
        os.chdir("/data/jantina/CoralNet/WAPA_Interns/masks")
        cv2.imwrite(img_name, mask)

        # load the mask and check the labels again
        # if they don't match there is a problem
        image = cv2.imread(os.path.join(img_name))
        print(img_name, np.unique(image))
    except:
        continue

2021-05-27-ASAN-A-T1 10.0-A (P5271395).png [ 35  37  78 118 120 255]
2021-05-27-ASAN-A-T1 10.0-A (P5271395).png [ 35  37  78 118 120 255]
2021-05-27-ASAN-A-T1 10.5-B (P5271453).png [ 37  78 120 131 255]
2021-05-27-ASAN-A-T1 10.5-B (P5271453).png [ 37  78 120 131 255]
2021-05-27-ASAN-A-T1 1.0-A (P5271377).png [ 37  44  78 118 120 255]
2021-05-27-ASAN-A-T1 1.0-A (P5271377).png [ 37  44  78 118 120 255]
2021-05-27-ASAN-A-T1 11.0-A (P5271397).png [ 37  68  78 101 103 118 120 255]
2021-05-27-ASAN-A-T1 11.0-A (P5271397).png [ 37  68  78 101 103 118 120 255]
2021-05-27-ASAN-A-T1 12.0-A (P5271399).png [ 31  37  78 101 120 131 255]
2021-05-27-ASAN-A-T1 12.0-A (P5271399).png [ 31  37  78 101 120 131 255]
2021-05-27-ASAN-A-T1 13.0-A (P5271401).png [ 37  44  78  87  91 118 120 255]
2021-05-27-ASAN-A-T1 13.0-A (P5271401).png [ 37  44  78  87  91 118 120 255]
2021-05-27-ASAN-A-T1 14.0-A (P5271403).png [ 31  37  78 118 120 255]
2021-05-27-ASAN-A-T1 14.0-A (P5271403).png [ 31  37  78 118 120 255]
2021

2021-05-27-ASAN-A-T4 4.0-A (P5271080).png [ 35  37  78 118 120 122 255]
2021-05-27-ASAN-A-T4 4.0-A (P5271080).png [ 35  37  78 118 120 122 255]
2021-05-27-ASAN-A-T4 4.5-B (P5271162).png [ 35  37  78 120 255]
2021-05-27-ASAN-A-T4 4.5-B (P5271162).png [ 35  37  78 120 255]
2021-05-27-ASAN-A-T4 5.0-A (P5271082).png [ 37  44  68  78 120 255]
2021-05-27-ASAN-A-T4 5.0-A (P5271082).png [ 37  44  68  78 120 255]
2021-05-27-ASAN-A-T4 5.5-B (P5271160).png [ 37 120 255]
2021-05-27-ASAN-A-T4 5.5-B (P5271160).png [ 37 120 255]
2021-05-27-ASAN-A-T4 6.0-A (P5271084).png [ 37  44  78 120 122 255]
2021-05-27-ASAN-A-T4 6.0-A (P5271084).png [ 37  44  78 120 122 255]
2021-05-27-ASAN-A-T4 6.5-B (P5271158).png [ 37  78 120 122 255]
2021-05-27-ASAN-A-T4 6.5-B (P5271158).png [ 37  78 120 122 255]
2021-05-27-ASAN-A-T4 7.0-A (P5271086).png [120 122 255]
2021-05-27-ASAN-A-T4 7.0-A (P5271086).png [120 122 255]
2021-05-27-ASAN-A-T4 7.5-B (P5271156).png [ 37  44  78 118 120 122 255]
2021-05-27-ASAN-A-T4 7.5-B (P527

2021-05-27-ASAN-B-T1 10.0-A (P5270787).png [  8  37  44  78  91 255]
2021-05-27-ASAN-B-T1 10.5-B (P5270845).png [ 37  78  91 255]
2021-05-27-ASAN-B-T1 10.5-B (P5270845).png [ 37  78  91 255]
2021-05-27-ASAN-B-T1 1.0-A (P5270769).png [ 30  37  61  78  91 255]
2021-05-27-ASAN-B-T1 1.0-A (P5270769).png [ 30  37  61  78  91 255]
2021-05-27-ASAN-B-T1 11.0-A (P5270789).png [ 37  61  78 255]
2021-05-27-ASAN-B-T1 11.0-A (P5270789).png [ 37  61  78 255]
2021-05-27-ASAN-B-T1 11.5-B (P5270843).png [ 37  78  85  95 255]
2021-05-27-ASAN-B-T1 11.5-B (P5270843).png [ 37  78  85  95 255]
2021-05-27-ASAN-B-T1 12.0-A (P5270791).png [ 35  37  78 255]
2021-05-27-ASAN-B-T1 12.0-A (P5270791).png [ 35  37  78 255]
2021-05-27-ASAN-B-T1 12.5-B (P5270841).png [ 27  37  78  91 131 255]
2021-05-27-ASAN-B-T1 12.5-B (P5270841).png [ 27  37  78  91 131 255]
2021-05-27-ASAN-B-T1 13.0-A (P5270793).png [ 37  72  78  91 255]
2021-05-27-ASAN-B-T1 13.0-A (P5270793).png [ 37  72  78  91 255]
2021-05-27-ASAN-B-T1 13.5-B (P5

2021-05-27-ASAN-B-T2 15.0-A (P5270899).png [ 35  37  61  78  91 118 120 255]
2021-05-27-ASAN-B-T2 15.0-A (P5270899).png [ 35  37  61  78  91 118 120 255]
2021-05-27-ASAN-B-T2 15.5-B (P5270937).png [ 37  91 106 118 120 255]
2021-05-27-ASAN-B-T2 15.5-B (P5270937).png [ 37  91 106 118 120 255]
2021-05-27-ASAN-B-T2 1.5-B (P5270965).png [ 35  37  61  78  91 106 116 118 120 255]
2021-05-27-ASAN-B-T2 1.5-B (P5270965).png [ 35  37  61  78  91 106 116 118 120 255]
2021-05-27-ASAN-B-T2 16.0-A (P5270901).png [ 37  78  79 106 118 120 255]
2021-05-27-ASAN-B-T2 16.0-A (P5270901).png [ 37  78  79 106 118 120 255]
2021-05-27-ASAN-B-T2 16.5-B (P5270935).png [ 37  91 101 116 118 120 255]
2021-05-27-ASAN-B-T2 16.5-B (P5270935).png [ 37  91 101 116 118 120 255]
2021-05-27-ASAN-B-T2 17.0-A (P5270903).png [ 26  78  91 101 116 118 120 255]
2021-05-27-ASAN-B-T2 17.0-A (P5270903).png [ 26  78  91 101 116 118 120 255]
2021-05-27-ASAN-B-T2 17.5-B (P5270933).png [ 35  37  61  78  79  91 118 120 255]
2021-05-27-AS

2021-11-06-AGAT-A-T2 16.0-A (PB061164).png [ 37  78  79 118 120 255]
2021-11-06-AGAT-A-T2 16.0-A (PB061164).png [ 37  78  79 118 120 255]
2021-11-06-AGAT-A-T2 17.0-A (PB061166).png [  9  13  16  37  38  78 116 118 120 255]
2021-11-06-AGAT-A-T2 17.0-A (PB061166).png [  9  13  16  37  38  78 116 118 120 255]
2021-11-06-AGAT-A-T2 18.0-A (PB061168).png [  9  13  37  38 101 120 255]
2021-11-06-AGAT-A-T2 18.0-A (PB061168).png [  9  13  37  38 101 120 255]
2021-11-06-AGAT-A-T2 19.0-A (PB061171).png [  9  10  13  16  37  38  78 120 122 255]
2021-11-06-AGAT-A-T2 19.0-A (PB061171).png [  9  10  13  16  37  38  78 120 122 255]
2021-11-06-AGAT-A-T2 20.0-A (PB061172).png [  9  13  16  37  38  78 118 120 255]
2021-11-06-AGAT-A-T2 20.0-A (PB061172).png [  9  13  16  37  38  78 118 120 255]
2021-11-06-AGAT-A-T2 2.0-A (PB061136).png [ 37  78  87 120 255]
2021-11-06-AGAT-A-T2 2.0-A (PB061136).png [ 37  78  87 120 255]
2021-11-06-AGAT-A-T2 21.0-A (PB061174).png [ 78 118 120 255]
2021-11-06-AGAT-A-T2 21.0

...

TRAINER NBUR 2020-08-10-AGAT-B-T5 1.5-B (P8104052).png [  9  13  16  22  37  38  61  85  91 120 122 126 255]
TRAINER NBUR 2020-08-10-AGAT-B-T5 5.5-B (P8104044).png [ 13  27  37  38  78  87 255]
TRAINER NBUR 2020-08-10-AGAT-B-T5 5.5-B (P8104044).png [ 13  27  37  38  78  87 255]
TRAINER PAST 2020-06-23-ADELUP-A-T1 15.0-A (P6230529).png [  8  16  17  37  61  62  78  79  99 116 118 120 255]
TRAINER PAST 2020-06-23-ADELUP-A-T1 15.0-A (P6230529).png [  8  16  17  37  61  62  78  79  99 116 118 120 255]
TRAINER PAST 2020-06-23-ADELUP-A-T1 21.0-A (P6230541).png [ 37  38  61  62  79  87 101 118 120 127 255]
TRAINER PAST 2020-06-23-ADELUP-A-T1 21.0-A (P6230541).png [ 37  38  61  62  79  87 101 118 120 127 255]
TRAINER PAST 2020-06-23-ADELUP-A-T1 23.0-A (P6230545).png [ 30  37  78  79  85  99 118 120 121 255]
TRAINER PAST 2020-06-23-ADELUP-A-T1 23.0-A (P6230545).png [ 30  37  78  79  85  99 118 120 121 255]
TRAINER PAST 2020-06-23-ADELUP-A-T1 24.0-A (P6230547).png [ 17  37  38  78  79  85  87 11

TRAINER RSTA 2020-06-23-ADELUP-A-T1 24.0-A (P6230547).png [ 14  37  38  78  87 120 255]
TRAINER RSTA 2020-06-23-ADELUP-A-T1 9.0-A (P6230517).png [ 35  37  39  78  87 118 120 255]
TRAINER RSTA 2020-06-23-ADELUP-A-T1 9.0-A (P6230517).png [ 35  37  39  78  87 118 120 255]
TRAINER RSTA 2020-06-23-ADELUP-A-T2 19.0-A (P6230646).png [ 13  14  37  38  78  87  97 118 120 255]
TRAINER RSTA 2020-06-23-ADELUP-A-T2 19.0-A (P6230646).png [ 13  14  37  38  78  87  97 118 120 255]
TRAINER RSTA 2020-06-23-ADELUP-A-T2 23.0-A (P6230654).png [ 37  78  87 101 102 116 120 255]
TRAINER RSTA 2020-06-23-ADELUP-A-T2 23.0-A (P6230654).png [ 37  78  87 101 102 116 120 255]
TRAINER RSTA 2020-06-23-ADELUP-A-T5 14.5-B (P6231016).png [ 37  38  87 101 118 120 126 255]
TRAINER RSTA 2020-06-23-ADELUP-A-T5 14.5-B (P6231016).png [ 37  38  87 101 118 120 126 255]
TRAINER RSTA 2020-06-23-ADELUP-A-T5 16.0-A (P6230968).png [ 31  37  38  78  87 121 255]
TRAINER RSTA 2020-06-23-ADELUP-A-T5 16.0-A (P6230968).png [ 31  37  38  78

TRAINER SBUT 2020-06-23-ADELUP-A-T5 20.0-A (P6230976).png [ 31  37  39  61  78  87 118 120 255]
TRAINER SBUT 2020-06-23-ADELUP-A-T5 20.0-A (P6230976).png [ 31  37  39  61  78  87 118 120 255]
TRAINER SBUT 2020-06-23-ADELUP-A-T5 24.0-A (P6230984).png [ 37  61  78  85  87  97 118 120 255]
TRAINER SBUT 2020-06-23-ADELUP-A-T5 24.0-A (P6230984).png [ 37  61  78  85  87  97 118 120 255]
TRAINER SBUT 2020-07-24-ASAN-B-T1 0.5-B (P7240126).png [ 35  37  39  61  78  91 255]
TRAINER SBUT 2020-07-24-ASAN-B-T1 0.5-B (P7240126).png [ 35  37  39  61  78  91 255]
TRAINER SBUT 2020-07-24-ASAN-B-T1 10.5-B (P7240106).png [ 37  61  78  85  91 118 255]
TRAINER SBUT 2020-07-24-ASAN-B-T1 10.5-B (P7240106).png [ 37  61  78  85  91 118 255]
TRAINER SBUT 2020-07-24-ASAN-B-T2 10.5-B (P7240215).png [ 27  35  37  78  91 106 120 129 255]
TRAINER SBUT 2020-07-24-ASAN-B-T2 10.5-B (P7240215).png [ 27  35  37  78  91 106 120 129 255]
TRAINER SBUT 2020-07-24-ASAN-B-T2 12.0-A (P7240153).png [ 20  27  37  61  78  91 101 1

TRAINER SYOK 2020-07-24-ASAN-B-T2 12.0-A (P7240153).png [ 20  30  37  61  78  91  99 101 116 255]
TRAINER SYOK 2020-07-24-ASAN-B-T2 13.0-A (P7240155).png [  8  37  85  91 106 116 255]
TRAINER SYOK 2020-07-24-ASAN-B-T2 13.0-A (P7240155).png [  8  37  85  91 106 116 255]
TRAINER SYOK 2020-07-24-ASAN-B-T2 15.0-A (P7240159).png [ 37  78  85  91 106 118 120 255]
TRAINER SYOK 2020-07-24-ASAN-B-T2 15.0-A (P7240159).png [ 37  78  85  91 106 118 120 255]
TRAINER SYOK 2020-07-24-ASAN-B-T2 17.0-A (P7240163).png [ 37  91 101 118 120 255]
TRAINER SYOK 2020-07-24-ASAN-B-T2 17.0-A (P7240163).png [ 37  91 101 118 120 255]
TRAINER SYOK 2020-07-24-ASAN-B-T2 24.0-A (P7240177).png [ 37  78 106 116 255]
TRAINER SYOK 2020-07-24-ASAN-B-T2 24.0-A (P7240177).png [ 37  78 106 116 255]
TRAINER SYOK 2020-07-24-ASAN-B-T2 8.0-A (P7240145).png [ 37  91 101 106 107 120 255]
TRAINER SYOK 2020-07-24-ASAN-B-T2 8.0-A (P7240145).png [ 37  91 101 106 107 120 255]
TRAINER SYOK 2020-07-24-ASAN-B-T2 9.5-B (P7240217).png [ 27 

TRAINER XQUI 2020-08-10-AGAT-A-T2 5.0-A (P8103085).png [ 38  79  87 118 120 255]
TRAINER XQUI 2020-08-10-AGAT-A-T2 5.0-A (P8103085).png [ 38  79  87 118 120 255]
TRAINER XQUI 2020-08-10-AGAT-A-T4 5.0-A (P8103305).png [ 38  79  85 120 122 255]
TRAINER XQUI 2020-08-10-AGAT-A-T4 5.0-A (P8103305).png [ 38  79  85 120 122 255]
TRAINER XQUI 2020-08-10-AGAT-B-T1 7.0-A (P8103531).png [  9  10  38  61  78  87 120 255]
TRAINER XQUI 2020-08-10-AGAT-B-T1 7.0-A (P8103531).png [  9  10  38  61  78  87 120 255]
TRAINER XQUI 2020-08-10-AGAT-B-T4 2.0-A (P8103846).png [  9  10  13  14  37  38  61 118 120 255]
TRAINER XQUI 2020-08-10-AGAT-B-T4 2.0-A (P8103846).png [  9  10  13  14  37  38  61 118 120 255]
TRAINER XQUI 2020-08-10-AGAT-B-T4 22.0-A (P8103886).png [ 27  37  38  61  79  85  87 118 120 255]
TRAINER XQUI 2020-08-10-AGAT-B-T4 22.0-A (P8103886).png [ 27  37  38  61  79  85  87 118 120 255]
TRAINER XQUI 2020-08-10-AGAT-B-T5 0.5-B (P8104054).png [  8   9  10  13  14  22  37  38  61  78  79  85 118 

## Rotating images

Some images might have tilted when being download, thus the annotations won't match. 

The message `These labels are out of bounds` will appear for these images.

These images will need to be rotated first !

In [ ]:
# go to root
os.chdir("/data/jantina/CoralNet/WAPA_RFM")

# take a single image
img_name = "2020-08-07-ADELUP-B-T3 2.5-B (P8072737).JPG"
image = io.imread(os.path.join('images', img_name))
print(image.shape)
plt.imshow(image)

# retrieve the annotations for that image
landmark = landmarks.iloc[img_names[img_names==img_name].index.values]
landmark = np.asarray(landmark)

# rotate
img = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
print(img.shape)
plt.imshow(img)

# replace the current image
os.chdir("/data/jantina/CoralNet/WAPA_RFM/images")
cv2.imwrite(img_name, img)

## Plot a single image with annotations

In [ ]:
# go to root
os.chdir("/data/jantina/CoralNet/WAPA_RFM")

# take a single image
img_name = "2020-08-07-ADELUP-B-T3 2.5-B (P8072737).JPG"

# retrieve the annotations for that image
landmarks = annotations.iloc[:, 1:3]
landmarks = landmarks.iloc[img_names[img_names==img_name].index.values]
landmarks = np.asarray(landmarks).astype('float').reshape(-1, 2)

def show_landmarks(image, landmarks):
    """Show image with landmarks"""
    plt.imshow(image)
    plt.scatter(landmarks[:, 1], landmarks[:, 0], s=10, marker='.', c='r')

# show the image with it's annotations
plt.figure()
show_landmarks(io.imread(os.path.join('images', img_name)),landmarks)
plt.show()